The Gans Project by Balu, Marvin, Frederico and Joachim 

In this project, our E-scooter company is trying to predict  effective locations in Germany to place our scooters. For the purpose of this project we are trying to attract air travellers and  tourists to use our commodity as a means of transportation. As an experiment we are limiting ourselves to 16 of the largest air hubs in Germany which are also popular tourist destinations. 

In [2]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import time
!pip install python-dotenv
%load_ext dotenv
%dotenv
import os
import pytz
from datetime import datetime
import mysql.connector
import sqlalchemy

The first task is to get the lat., long., population, country, countrycode and wiki_dataid for a list of cities in Germany and put it in a dataframe. This code is designed so that in future more cities maybe added.

In [ ]:
#API_key = os.getenv("API_K_C")
 #gets the API key from .env
#define the list of cities

citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']

#strategy is to first get the wikidataid and than collect the data from geoDB; openweather has no population data or at least I coul´nt see them 


API_key = os.getenv("API_K_C")
print(API_key)
def demo(cities):
    cities_id = [] # initiate an empty id list
    dfList = []
    for city in cities:
        #retrieve the wikidataId
        time.sleep(0)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) #go to the wiki site of the city
        citem = requests.get(url1, 'html.parser') # get the html
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)# soup the content
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] #find the link that contains the wikidataid e.g. London https://www.wikidata.org/wiki/Q84
        #wl.append(wikidata_link)
        # \d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits
        #for group() in re see: https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
        #use the wikidataId to retrieve infrormation from geoDB
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)# gets a json-like string fromg geoDB containing the necessary informations
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
    df_demo = pd.DataFrame(dfList) # transform the list to df
        #print(dfList)
    df_demo.to_csv('./City')
    return df_demo
demo(citd) #run the whole thing - good luck!

Focussing on these 15 cities, we are collecting the weather forecast. This DF is subject to change as the weather information is updated as needed.

In [30]:
citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']#alternatively only German cities

def weather(cities):
    API_key = os.getenv("API_K_W") #gets key from the env file

    tz = pytz.timezone('Europe/Berlin')# timezone needs to be imported and translated for the european cities (Europe_Berlin to Europe/Berlin), see cell below
    now = datetime.now().astimezone(tz)#for the timestamp time of sampling

    a = {'City': [], 'Country': [], 'ForecastTime': [], 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []} #usefull form of a dic, gives the right form when change to df, see cell below 

    for city in cities: #runs a loop over the cities
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        response = requests.get(url)
        json = response.json() # the first 'priciple' key for this json is 'list', which is not good practice in my op but must be used to loop over it
    #print(json['city']['name'])
    # print(type(json))

        for i in json['list']:# informations will be added to the dic a according to the json structure (see weimar cell above), if-else for rain does not work (is there or not but python cannot know that it could be there), therefore try except - appears to work but I´m not sure if clean
          #print(i)  
            a['City'].append(json['city']['name'])
            a['Country'].append(json['city']['country'])
            a['ForecastTime'].append(i['dt_txt'])
            a['Outlook'].append(i['weather'][0]['description'])
            a['Temperature'].append(i['main']['temp'])
            a['Clouds'].append(i['clouds']['all'])
            try:
                a['Rain'].append(i['rain']['3h'])
            except:
                a['Rain'].append('0')
            try:
                a['Snow'].append(i['snow']['3h'])
            except:
                a['Snow'].append('0')
            a['WindSpeed'].append(i['wind']['speed'])
            a['DateInformation'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
    b=pd.DataFrame(a) #turns a to datafram , see cell below
    b.to_csv('./Weather')
    return b
    
weather(citd)


,City,Country,ForecastTime,Outlook,Temperature,Clouds,Rain,Snow,WindSpeed,DateInformation
0,Baden-Baden,DE,2022-06-15 12:00:00,clear sky,26.58,2,0,0,2.51,15/06/2022 12:09:46
1,Baden-Baden,DE,2022-06-15 15:00:00,clear sky,27.73,0,0,0,1.53,15/06/2022 12:09:46
2,Baden-Baden,DE,2022-06-15 18:00:00,clear sky,25.32,0,0,0,0.47,15/06/2022 12:09:46
3,Baden-Baden,DE,2022-06-15 21:00:00,clear sky,20.56,0,0,0,2.64,15/06/2022 12:09:46
4,Baden-Baden,DE,2022-06-16 00:00:00,clear sky,19.22,3,0,0,1.92,15/06/2022 12:09:46
...,...,...,...,...,...,...,...,...,...,...
635,Stuttgart,DE,2022-06-19 21:00:00,light rain,23.77,12,0.19,0,2.58,15/06/2022 12:09:46
636,Stuttgart,DE,2022-06-20 00:00:00,moderate rain,16.66,45,3.3,0,1.77,15/06/2022 12:09:46
637,Stuttgart,DE,2022-06-20 03:00:00,light rain,16.03,67,0.85,0,2.41,15/06/2022 12:09:46
638,Stuttgart,DE,2022-06-20 06:00:00,light rain,18.72,70,0.89,0,4.94,15/06/2022 12:09:46


For the cities in our database we now get the details of the airports. 

In [ ]:
#citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']
ac = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
airport = []
API_KE = '0515339d6fmsh81d78dce0cb28cap1cbb53jsnb31e102cdb8c'

for a in ac:
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{a}"

    querystring = {"withRunways":"true"}

    headers = {
        "X-RapidAPI-Key": API_KE,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    airport.append(response.json())
    
print(airport)
   # listCityShort[a]=response.json()['city']
    #print(listCityShort)

    

In [37]:
from IPython.display import JSON
#citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']
ac = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']

def airp(cities):
    airport = []
    API_KE = '1821798fe1mshc0aaf1be2c43cedp1289ecjsn546ec73b6483'

    for i in ac:
        url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{i}"

        querystring = {"withRunways":"true"}

        headers = {
            "X-RapidAPI-Key": API_KE,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        aa = {}#make a dictionary to retrieve the information
        aa['City'] = response.json()['municipalityName']
        aa['icao'] = response.json()['icao']
        aa['iata'] = response.json()['iata']
        aa['Time_Zone'] = response.json()['timeZone']
        aa['Country'] = response.json()['country']['name']
        aa['Country_Code'] = response.json()['country']['code']
        aa['Continent'] = response.json()['continent']['name']
        try:
            aa['Website'] = response.json()['urls']['webSite']              
        except:
            aa['Website'] = 'Not available'
        airport.append(aa)
        
    b = pd.DataFrame(airport)
    
    b.to_csv('./Airports')
    return b 

airp(ac)


,City,icao,iata,Time_Zone,Country,Country_Code,Continent,Website
0,Baden-Baden,EDSB,FKB,Europe/Berlin,Germany,DE,Europe,http://www.baden-airpark.de/startseite.html
1,Berlin,EDDT,TXL,Europe/Berlin,Germany,DE,Europe,http://www.berlin-airport.de/en/travellers-txl...
2,Berlin,EDDB,BER,Europe/Berlin,Germany,DE,Europe,https://ber.berlin-airport.de/
3,Cologne,EDDK,CGN,Europe/Berlin,Germany,DE,Europe,http://www.koeln-bonn-airport.de/en/
4,Bremen,EDDW,BRE,Europe/Berlin,Germany,DE,Europe,http://www.bremen-airport.com/
5,Dresden,EDDC,DRS,Europe/Berlin,Germany,DE,Europe,https://www.dresden-airport.de
6,Dortmund,EDLW,DTM,Europe/Berlin,Germany,DE,Europe,Not available
7,Duesseldorf,EDDL,DUS,Europe/Berlin,Germany,DE,Europe,https://www.dus.com/en
8,Frankfurt-am-Main,EDDF,FRA,Europe/Berlin,Germany,DE,Europe,http://www.frankfurt-airport.de/
9,Hamburg,EDDH,HAM,Europe/Berlin,Germany,DE,Europe,https://www.hamburg-airport.de/en/


For all the airports in our list we can have the  details of the arriving flights. The most important detail being the flight times. The list may also optionally include the details of the flights origin as in future we can have a statistical analysis of the passengers who may use our scooters. This is also a changing DF and with queries we can find out the most and least busiest times of the day. This code is not ideal since it does not show the destination airport. Need to edit the codes.

In [33]:
from IPython.display import JSON

API_key = '1821798fe1mshc0aaf1be2c43cedp1289ecjsn546ec73b6483'
url = f"https://aerodatabox.p.rapidapi.com/airports/icao/EDSB"
querystring = {"withRunways":"true"}
headers = {
    "X-RapidAPI-Key": API_key,
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

# url = (f"http://api.openweathermap.org/data/2.5/forecast?q=Baden-Baden&appid={API_key}&units=metric")
# response = requests.get(url)
# #json = response.json() 
responses = requests.request("GET", url, headers=headers, params=querystring)
# #print(type(responses.json()))
draft = responses.json()
JSON(draft)

<IPython.core.display.JSON object>

In [36]:
import os

API_KE = '3cd15bf266msh2331a2a034ea490p1c96a2jsn828e9dce3a50' #os.getenv("API_C_F")
#print(API_KE)
cit = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
def flights(cities):
    
    a = {'City': [], 'FlightNumber': [], 'FlightStatus': [], 'OriginAirport': [], 'ScheduledTime': [], 'AirLine': []}#, 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []}
    for city in cities:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/2022-06-08T06:00/2022-06-08T17:00"
        querystring = {"direction":"Arrival","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}
        headers = {
           "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
           "X-RapidAPI-Key": API_KE
           }
        responses = requests.request("GET", url, headers=headers, params=querystring)
        json = responses.json()
        
        for i in json['arrivals']:
            a['City'] = city
            a['FlightNumber'].append(i['number'])
            a['FlightStatus'].append(i['status'])
            a['OriginAirport'].append(i['movement']['airport']['name'])
            a['ScheduledTime'].append(i['movement']['scheduledTimeLocal'])
            a['AirLine'].append(i['airline']['name'])
        
    c = pd.DataFrame(a)
    c.to_csv('./Flights')
    return c
flights(cit)



,City,FlightNumber,FlightStatus,OriginAirport,ScheduledTime,AirLine
0,EDDS,EW 6804,Unknown,Palma De Mallorca,2022-06-08 10:25+02:00,Eurowings
1,EDDS,FR 5183,Unknown,Palma De Mallorca,2022-06-08 11:30+02:00,Ryanair
2,EDDS,GFD 41,Approaching,Unknown,2022-06-08 11:49+02:00,GFD
3,EDDS,W6 6617,Unknown,Tirana,2022-06-08 13:30+02:00,Wizz Air
4,EDDS,FR 9211,Approaching,Girona,2022-06-08 15:50+02:00,Ryanair
...,...,...,...,...,...,...
2480,EDDS,LH 2148,Arrived,Munich,2022-06-08 16:45+02:00,Lufthansa
2481,EDDS,TI 329,Arrived,Antalya,2022-06-08 16:30+02:00,Tailwind
2482,EDDS,EW 4007,Arrived,Palma De Mallorca,2022-06-08 15:25+02:00,Eurowings
2483,EDDS,EW 2791,Arrived,Burgas,2022-06-08 15:40+02:00,Eurowings


In [ ]:
demo(citd)

In [ ]:
weather(citd)

In [ ]:
flights(cit)

In [ ]:
airp(ac)

After completing all the DFs, they are exported to an already created SQL Schema. Note: No tables were created in MySQL workbench all tables are created from here. Once created go to sql to create Primary and Foreign Keys. Also the flights DF is not exported as the city column is missing and I have ran out of API calls.

Please feel free to alter the codes in anyway that seem fit.

In [50]:

# define connection details
cnx = mysql.connector.connect(
    user='root',
    password='Majupau7##1.0', #type your root password here
    host='127.0.0.1', # to connect to your local instance
    database='scooter' #type the name of the database you want to use here
)
 
# connect to database
cursor = cnx.cursor()
schema="scooter"
host="127.0.0.1"
user="root"
password="Majupau7##1.0"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
# !pip install pymysql
# weather(citd).to_sql('weather', 
#               if_exists='append', 
#               con=con, 
#               index=False)
# airp(ac).to_sql('airports', 
#               if_exists='append', 
#               con=con, 
#               index=False)
# demo(citd).to_sql('cities', 
#               if_exists='append', 
#               con=con, 
#               index=False)
# flights(cit).to_sql('flights', 
#               if_exists='append', 
#               con=con, 
#               index=False)